In [1]:
import numpy as np
from hmm_utils import HMM
from attackers.smoothing_state_attacker import ss_attacker
from attackers.decoding_attacker import dec_attacker
from params import *
from solvers.mc_ennumeration import MC_enumeration
from solvers.simulated_annealing import simulated_annealing
#from solvers.aps_gibbs import aps_gibbs
from solvers.nn_RS.nn_RS import *
#from solvers.aps_gibbs_class import *

from solvers.aps_gibbs_class_exp_decay import *

from scipy.special import softmax

In [2]:
priors     = np.array([0.5,0.3,0.2])
transition = np.array([[0.85, 0.05,0.1],
                       [0.05, 0.9,0.05],
                        [1/2, 1/4, 1/4]])

emission   = np.array([[0.699, 0.05, 0.1, 0.05, 0.1, 0.001],
                    [0.001, 0.1, 0.1, 0.299, 0.3, 0.2],
                       [0.1, 0.2, 0.1, 0.2, 0.1, 0.3]])

rho_probs = np.ones(6)

x_vector = np.array([ [5], [3], [4], [3], [5] ])

In [3]:
hmm = HMM(n_components=3, n_obs=6 )
hmm.emissionprob_ = emission
hmm.transmat_     = transition
hmm.startprob_    = priors

In [4]:
V, seq = hmm.nu(x_vector)
seq

array([1, 1, 1, 1, 1])

In [5]:
attacker = ss_attacker(priors, transition, emission, rho_probs,
         x_vector, w1 = 1.0 ,w2 = 0.0 , t = 3, state= 0, c=1, k_value=100000)

attacker = dec_attacker(priors, transition, emission, rho_probs,
         x_vector, w1 = 10.0 ,w2 = 0.0 , seq = np.array([2,2,2,2,2]), k_value=100000)

Z_set = attacker.generate_attacks()

# Full ennumeration

In [ ]:
z_star, z_vec = MC_enumeration(attacker, 10)

In [ ]:
np.argsort(-z_vec)[:10]

In [9]:
attacker.expected_utility(Z_set[0], 10000)

2.989512134961151

In [21]:
Z_set[0]

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

# Simulated annealing

In [32]:
z_star = simulated_annealing(attacker, 200, verbose=True)

Percentage completed: 0.0
Current state [[0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]
Percentage completed: 25.0
Current state [[0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
Percentage completed: 50.0
Current state [[0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]
Percentage completed: 75.0
Current state [[0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [24]:
attacker.expected_utility(z_star, N=10000)

6.489566960587724

In [33]:
attack_obs = attacker.attack_X(np.ones_like(z_star), z_star)
V, seq = hmm.nu(attack_obs.astype(int))

In [34]:
seq

array([2, 0, 2, 2, 2])

# Gibbs - APS

In [6]:
aps = aps_gibbs(attacker)

In [7]:
z_star, quality = aps.iterate(simulation_seconds=20)

1
1
3
6
11
20
36
66
121
221
403
735
1339
2440
4447


In [8]:
quality

55.8971030788679

In [9]:
attack_obs = attacker.attack_X(np.ones_like(z_star), z_star)
V, seq = hmm.nu(attack_obs.astype(int))
seq

array([2, 0, 2, 2, 2])

# RS with Neural Network

In [16]:
rs = nn_RS(attacker, RS_iters=5000, mcts_iters=10, eps=0.05, lr=0.005)

In [17]:
z_star, quality = rs.iterate(simulation_seconds=20)

In [18]:
quality

55.89645814832329

In [19]:
attack_obs = attacker.attack_X(np.ones_like(z_star), z_star)
V, seq = hmm.nu(attack_obs.astype(int))
seq

array([2, 0, 2, 2, 0])

# Full APS - Old

In [197]:
def propose_z(z):
    z_new = z.copy()
    idx = np.random.choice(np.arange(z.shape[0]))
    candidates  = np.diag( np.ones(z.shape[1]) )
    candidates  = candidates[~np.all(candidates == z[idx], axis=1)]

    z_new[idx] = candidates[ np.random.choice(np.arange(candidates.shape[0])) ]
    return z_new

def update_z(H, hmm, z_new, z_old, old_value, k_value):
    
    transtion_mat_list = hmm.sample_mat(hmm.transmat_, n = H, k = k_value)
    emission_mat_list = hmm.sample_mat(hmm.emissionprob_, n = H, k = k_value)
    priors_vec_list = hmm.sample_mat(np.array([list(hmm.startprob_)]), n = H, k = k_value)
    
    hmms = []
    rho_matrix_samples = []
    
    value = 0
    
    for h in range(H):
        
        hmm_n = HMM(hmm.n_components, hmm.n_obs)
        hmm_n.startprob_ = priors_vec_list[h].reshape(-1)
        hmm_n.transmat_ = transtion_mat_list[h]
        hmm_n.emissionprob_ = emission_mat_list[h]
        rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)
        
        hmms.append(hmm_n)
        rho_matrix_samples.append(rho_matrix)
        
        y_t = hmm_n.attack_X(X = x_vector, z_matrix = z_new, 
                             rho_matrix = rho_matrix).astype('int')
        
        value += np.log( 
            attacker.utility(hmm = hmm_n, z_matrix = z_new, 
                             x_obs_vector = x_vector, y_t = y_t) + 6 )
        
    value /= H

    prob =  np.exp(H*value - H*old_value)

    
    if np.random.uniform() < prob:
        
        print("Accept!")
        print(prob)
        print(z_new)
        print(z_old)
        
        return z_new, value
    
    else:
        
        return z_old, old_value

In [211]:
z_init = Z_set[ np.random.choice(Z_set.shape[0]) ]

z_init =np.array([[0., 1., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.]])


hh = 15000
k_value = 1000000
##
transtion_mat_list = hmm.sample_mat(hmm.transmat_, n = hh, k = k_value)
emission_mat_list = hmm.sample_mat(hmm.emissionprob_, n = hh, k = k_value)
priors_vec_list = hmm.sample_mat(np.array([list(hmm.startprob_)]), n = hh, k = k_value)
rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)

hmms = []
rho_matrix_samples = []

value = 0
for temp in range(hh):

    hmm_n = HMM(hmm.n_components, hmm.n_obs)
    hmm_n.startprob_ = priors_vec_list[temp].reshape(-1)
    hmm_n.transmat_ = transtion_mat_list[temp]
    hmm_n.emissionprob_ = emission_mat_list[temp]
    rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)

    hmms.append(hmm_n)
    rho_matrix_samples.append(rho_matrix)

    y_t = hmm_n.attack_X(X = x_vector, z_matrix = z_init, rho_matrix = rho_matrix).astype('int')
    value += np.log( 
        attacker.utility(hmm = hmm_n, z_matrix = z_init, x_obs_vector = x_vector, y_t = y_t) + 6 )
    

value /= hh
value   


1.9443352596947014

In [212]:
for temp in np.arange(15000, 20000, 250):

    z_new = propose_z(z_init)
    z_init, value = update_z(temp, hmm, z_new, z_init, value, k_value)
    
    print(temp)
    print(z_init)

   


15000
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
15250
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
15500
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
Accept!
1.524466408672834
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
15750
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
16000
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
16250
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
16500
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]


KeyboardInterrupt: 